# 영화 평점 분석 실습

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

## 1. 영화 평점 데이터 적재 및 전처리

In [2]:
# 사용자 데이터 읽어오기
users = pd.read_csv('data/movielens/users.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '성별','연령','직업','지역'])
users.head()

,사용자아이디,성별,연령,직업,지역
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [3]:
# 평점 데이터 읽어오기
ratings = pd.read_csv('data/movielens/ratings.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '영화아이디','평점','타임스탬프'])
ratings.head()

,사용자아이디,영화아이디,평점,타임스탬프
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 영화데이터 읽어오기
movies = pd.read_csv('data/movielens/movies.dat', sep = '::', engine = 'python',
                   names = ['영화아이디','영화제목','장르'], encoding = 'latin-1')
movies.head()

,영화아이디,영화제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
# 데이터 개수 
print("사용자수: ", len(users))
print("평점 수: ", len(ratings))
print("영화 수: ", len(movies))

사용자수:  6040
평점 수:  1000209
영화 수:  3883


In [15]:
# 평점을 한 번도 안 준 사용자가 있는가? 
ratings.사용자아이디.nunique() # 중복을 제거하고 유니크한 값을 알려준다.

6040

##### 평점을 한번도 안 준 사용자는 없다.

In [16]:
# 평점이 하나도 없는 영화가 있는가? 
ratings.영화아이디.nunique()

3706

###### 있다. 영화 수는 3883개인데, 3706개가 나왔기 때문에. 
###### 3883 - 3706 = 177개의 영화는 평점이 하나도 없다.

In [17]:
#3개의 데이터프레임을 하나로 합치기
data = pd.merge(users, ratings)
data = pd.merge(data, movies)

In [18]:
data.head()

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


## 2. 보고 싶은 영화 찾기
영화들의 평점 평균을 구하여, 사람들에게 인정받는 (평점이 높은) 영화 찾기

In [24]:
# 영화제목이 중복되는 것이 있는지 체크하자 
# 영화제목이 유니크한 개수와 영화아이디가 유니크한 개수를 비교하자
movies.nunique()

영화아이디    3883
영화제목     3883
장르        301
dtype: int64

###### 영화아이디 개수 = 영화제목 -> 영화제목이 중복되는 경우는 없다.
###### 만약 중복되는 것이 있다면, 영화아이디를 사용해야 한다.
###### 애초에 아이디랑 제목을 같이 써주면 중복을 구별할 수 있을 것이다

In [28]:
# 영화들의 평점 평균을 구하여, 평점이 높은 영화 찾기 
data.pivot_table(index='영화제목' ,aggfunc = 'mean', values='평점')

,평점
영화제목,
"$1,000,000 Duck (1971)",3.027027
'Night Mother (1986),3.371429
'Til There Was You (1997),2.692308
"'burbs, The (1989)",2.910891
...And Justice for All (1979),3.713568
...,...
"Zed & Two Noughts, A (1985)",3.413793
Zero Effect (1998),3.750831
Zero Kelvin (Kjærlighetens kjøtere) (1995),3.500000


In [29]:
data.pivot_table(index='영화제목' ,aggfunc = 'mean', values='평점').nlargest(5, '평점')

,평점
영화제목,
"Baby, The (1973)",5.0
Bittersweet Motel (2000),5.0
Follow the Bitch (1998),5.0
"Gate of Heavenly Peace, The (1995)",5.0
Lured (1947),5.0


##### 만점? => 평점의 개수가 적은 경우에 가능하다. 
###### 때문에 mean, count 함수를 같이 적용해준다.

In [30]:
data.pivot_table(index='영화제목' ,aggfunc = ['mean','count'], values='평점')

,mean,count
,평점,평점
영화제목,,
"$1,000,000 Duck (1971)",3.027027,37
'Night Mother (1986),3.371429,70
'Til There Was You (1997),2.692308,52
"'burbs, The (1989)",2.910891,303
...And Justice for All (1979),3.713568,199
...,...,...
"Zed & Two Noughts, A (1985)",3.413793,29
Zero Effect (1998),3.750831,301


In [31]:
data.pivot_table(index='영화제목' ,aggfunc = ['mean','count'], values='평점').nlargest(10, ('mean','평점'))

,mean,count
,평점,평점
영화제목,,
"Baby, The (1973)",5.0,1
Bittersweet Motel (2000),5.0,1
Follow the Bitch (1998),5.0,1
"Gate of Heavenly Peace, The (1995)",5.0,3
Lured (1947),5.0,1
One Little Indian (1973),5.0,1
Schlafes Bruder (Brother of Sleep) (1995),5.0,1
Smashing Time (1967),5.0,2


In [ ]:
# 평점의 평균이 4.5 이상이고 평점의 개수도 1000개 이상인 영화를 보고싶은 영화로 선정하겠다.

평균 평점이 만점인 영화들이 최상위에 위치함. 
일반적으로 평점이 만점인 경우는 대부분 평점의 개수가 매우 적은 경우이므로, 이를 확인하기 위해 평점의 개수도 함께 구해본다. 

In [35]:
ex = data.pivot_table(index='영화제목' ,aggfunc = ['mean','count'], values='평점')
ex.columns = ['평균','개수']
ex

,평균,개수
영화제목,,
"$1,000,000 Duck (1971)",3.027027,37
'Night Mother (1986),3.371429,70
'Til There Was You (1997),2.692308,52
"'burbs, The (1989)",2.910891,303
...And Justice for All (1979),3.713568,199
...,...,...
"Zed & Two Noughts, A (1985)",3.413793,29
Zero Effect (1998),3.750831,301
Zero Kelvin (Kjærlighetens kjøtere) (1995),3.500000,2


In [386]:
ex[(ex.평균 >= 4.0) & (ex.개수 >= 1000)]

,평균,개수
영화제목,,
2001: A Space Odyssey (1968),4.068765,1716
"African Queen, The (1951)",4.251656,1057
Alien (1979),4.159585,2024
Aliens (1986),4.125824,1820
Amadeus (1984),4.251809,1382
...,...,...
"Untouchables, The (1987)",4.007986,1127
"Usual Suspects, The (1995)",4.517106,1783
When Harry Met Sally... (1989),4.073342,1568


## [실습 #1] 여자들이 좋아하는 영화 찾기 
### - 여성 평점이 4.0 이상이고 여성 평점의 개수가 500개 이상인 영화

# 정답

In [312]:
ex1 = data[data.성별 =='F'].pivot_table(index='영화제목', values='평점',aggfunc=['mean','count'])

In [314]:
ex1 = data.pivot_table(index='영화제목',columns='성별',values='평점', aggfunc=['mean','count'])

In [315]:
ex1.xs('F', axis=1, level=1)

,mean,count
영화제목,,
"$1,000,000 Duck (1971)",3.375000,16.0
'Night Mother (1986),3.388889,36.0
'Til There Was You (1997),2.675676,37.0
"'burbs, The (1989)",2.793478,92.0
...And Justice for All (1979),3.828571,35.0
...,...,...
"Zed & Two Noughts, A (1985)",3.500000,8.0
Zero Effect (1998),3.864407,59.0
Zero Kelvin (Kjærlighetens kjøtere) (1995),NaN,NaN


In [322]:
여성인기영화 = (ex1['mean']>=4.0) & (ex1['count']>=500)

# =======================================+

In [139]:
data.sort_values('평점')

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르
900153,268,F,18,12,29708,2793,1,976648645,"American Werewolf in Paris, An (1997)",Comedy|Horror
871773,2618,F,18,4,53538,1970,1,973659020,"Nightmare on Elm Street 3: Dream Warriors, A (...",Horror
871769,2523,M,25,17,49423,1970,1,974064502,"Nightmare on Elm Street 3: Dream Warriors, A (...",Horror
922310,5761,M,25,20,90048,166,1,982013231,"Doom Generation, The (1995)",Comedy|Drama
958833,4072,M,50,6,93111,3819,1,965451714,Tampopo (1986),Comedy
...,...,...,...,...,...,...,...,...,...,...
291660,4796,M,35,7,98103,2890,5,962986218,Three Kings (1999),Drama|War
291661,4800,M,18,4,80521,2890,5,962943784,Three Kings (1999),Drama|War
761513,5634,M,25,14,55406,1663,5,959010654,Stripes (1981),Comedy
291656,4773,M,45,1,22314,2890,5,963067067,Three Kings (1999),Drama|War


In [96]:
fm =data[data.성별=='F']

In [195]:
fm=fm.pivot_table(index=['영화제목'],values='평점',aggfunc=['mean','count']).swaplevel(1,0,axis=1)

In [120]:
fm[(fm[('평점','mean')]>=4.5) & (fm[('평점','count')]>=500)]

평점      
                                      mean count
영화제목                                            
Schindler's List (1993)           4.562602   615
Shawshank Redemption, The (1994)  4.539075   627

## [실습 #2] 실습 #1에서 구한 영화(여성인기영화)의 장르를 분석해 보자.
여성인기영화의 장르 통계 구하기

예를 들어, 여성인기영화 중 Drama 장르의 영화는 10개, Action 영화는 3개, ...

# 정답

### 장르정보 가져오기

In [318]:
movies # 장르정보 가져오기 

,영화아이디,영화제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [319]:
여성인기영화.index

Index(['$1,000,000 Duck (1971)', ''Night Mother (1986)',
       ''Til There Was You (1997)', ''burbs, The (1989)',
       '...And Justice for All (1979)', '1-900 (1994)',
       '10 Things I Hate About You (1999)', '101 Dalmatians (1961)',
       '101 Dalmatians (1996)', '12 Angry Men (1957)',
       ...
       'Young Poisoner's Handbook, The (1995)', 'Young Sherlock Holmes (1985)',
       'Young and Innocent (1937)', 'Your Friends and Neighbors (1998)',
       'Zachariah (1971)', 'Zed & Two Noughts, A (1985)', 'Zero Effect (1998)',
       'Zero Kelvin (Kjærlighetens kjøtere) (1995)', 'Zeus and Roxanne (1997)',
       'eXistenZ (1999)'],
      dtype='object', name='영화제목', length=3706)

In [323]:
# isin() 함수 활용 
movies[movies.영화제목.isin(여성인기영화.index)]

,영화아이디,영화제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [325]:
# 여성인기영화와 movies 데이터를 합치는 방법 
# 2-1) merge 
pd.merge(여성인기영화.reset_index(), movies)


,영화제목,F,M,영화아이디,장르
0,"$1,000,000 Duck (1971)",False,False,2031,Children's|Comedy
1,'Night Mother (1986),False,False,3112,Drama
2,'Til There Was You (1997),False,False,779,Drama|Romance
3,"'burbs, The (1989)",False,False,2072,Comedy
4,...And Justice for All (1979),False,False,3420,Drama|Thriller
...,...,...,...,...,...
3701,"Zed & Two Noughts, A (1985)",False,False,3223,Drama
3702,Zero Effect (1998),False,False,1845,Comedy|Thriller
3703,Zero Kelvin (Kjærlighetens kjøtere) (1995),False,False,1364,Action
3704,Zeus and Roxanne (1997),False,False,1426,Children's


In [392]:
여성인기영화

성별,F,M
영화제목,,
"$1,000,000 Duck (1971)",False,False
'Night Mother (1986),False,False
'Til There Was You (1997),False,False
"'burbs, The (1989)",False,False
...And Justice for All (1979),False,False
...,...,...
"Zed & Two Noughts, A (1985)",False,False
Zero Effect (1998),False,False
Zero Kelvin (Kjærlighetens kjøtere) (1995),False,False


In [332]:
# 2-2) concat은 행을 기준으로 합치기 때문에 movie 영화제목을 행으로 넣다 
ex2 = pd.concat([여성인기영화, movies.set_index('영화제목')], axis=1, join='inner').장르
# inner를 써줌으로써 NaN 빼버림

In [337]:
ex2_expand = ex2.str.split('|', expand = True)

In [343]:
ex2_expand

,0,1,2,3,4,5
영화제목,,,,,,
"$1,000,000 Duck (1971)",Children's,Comedy,None,None,None,None
'Night Mother (1986),Drama,None,None,None,None,None
'Til There Was You (1997),Drama,Romance,None,None,None,None
"'burbs, The (1989)",Comedy,None,None,None,None,None
...And Justice for All (1979),Drama,Thriller,None,None,None,None
...,...,...,...,...,...,...
"Zed & Two Noughts, A (1985)",Drama,None,None,None,None,None
Zero Effect (1998),Comedy,Thriller,None,None,None,None
Zero Kelvin (Kjærlighetens kjøtere) (1995),Action,None,None,None,None,None


In [359]:
# count 세기 
ex2_expand[1].value_counts()

1824

In [356]:
#장르1에 장르2더하고 그 결과에 장르3더하고,.... 
여성인기장르 = Series(dtype = 'float64')
for col in ex2_expand.columns:
    장르 =ex2_expand[col].value_counts()
    여성인기장르 = 여성인기장르.add(장르, fill_value=0)

In [357]:
여성인기장르

Action          495.0
Adventure       281.0
Animation       105.0
Children's      250.0
Comedy         1163.0
Crime           201.0
Documentary     110.0
Drama          1493.0
Fantasy          68.0
Film-Noir        44.0
Horror          339.0
Musical         113.0
Mystery         104.0
Romance         459.0
Sci-Fi          274.0
Thriller        485.0
War             141.0
Western          67.0
dtype: float64

# =============================================

In [275]:
jr = data[data.성별=='F']

In [236]:
a = "Animation|Children|Comedy" 

In [249]:
a.split(sep='|')

['Animation', 'Children', 'Comedy']

In [282]:
new_jr = jr.pivot_table(index=['장르'], values='영화제목',aggfunc='count')

In [286]:
new_jr = new_jr.reset_index()

In [295]:
ans = []

In [296]:
for i in list(range(0,len(new_jr))):
    ans.append(new_jr.장르[i].split(sep='|'))
    

In [291]:
type(ans)

list

TypeError: unhashable type: 'list'

## [실습 #3] 남자와 여자의 호불호가 크게 갈리는 영화 10개 찾기
전체 평점의 개수가 500개 이상인 영화만 대상으로 함.

# 정답

In [361]:
ex3 = data.pivot_table(index='영화제목',columns='성별',values='평점',aggfunc=['mean','count'])

In [365]:
# 1. 전체 평점의 개수가 500개 이상인 영화만 선택 
ex3 = ex3[(ex3[('count','F')] + ex3[('count','M')]) >= 500]

In [370]:
# 2. 남자평점과 여자평점의 차이 계산 
ex3['diff'] = abs(ex3[('mean','F')] - ex3[('mean','M')])

In [372]:
ex3.nlargest(10, 'diff')

mean            count         \
성별                                             F         M      F      M   
영화제목                                                                       
Dirty Dancing (1987)                    3.790378  2.959596  291.0  396.0   
Good, The Bad and The Ugly, The (1966)  3.494949  4.221300   99.0  723.0   
Dumb & Dumber (1994)                    2.697987  3.336595  149.0  511.0   
Evil Dead II (Dead By Dawn) (1987)      3.297297  3.909283   74.0  474.0   
Grease (1978)                           3.975265  3.367041  283.0  534.0   
Caddyshack (1980)                       3.396135  3.969737  207.0  760.0   
Animal House (1978)                     3.628906  4.167192  256.0  951.0   
Exorcist, The (1973)                    3.537634  4.067239  186.0  699.0   
Rocky Horror Picture Show, The (1975)   3.673016  3.160131  315.0  918.0   
Big Trouble in Little China (1986)      2.987952  3.485030   83.0  501.0   

                                            diff  
성별                                                
영화제목                                              
Dirty Dancing (1987)                    0.830782  
Good, The Bad and The Ugly, The (1966)  0.726351  
Dumb & Dumber (1994)                    0.638608  
Evil Dead II (Dead By Dawn) (1987)      0.611985  
Grease (1978)                           0.608224  
Caddyshack (1980)                       0.573602  
Animal House (1978)                     0.538286  
Exorcist, The (1973)                    0.529605  
Rocky Horror Picture Show, The (1975)   0.512885  
Big Trouble in Little China (1986)      0.497078

# ===========================================

In [126]:
m = data[data.성별=='M']

In [127]:
m = m.pivot_table(index='영화제목',values='평점', aggfunc=['count', 'mean']).swaplevel(1,0, axis=1)

In [149]:
new_fm = fm[fm[('평점','count')]>=500]

In [161]:
new_m = m[m[('평점','count')]>=500]

In [166]:
new_m = new_m.add_prefix('남성_')

In [167]:
new_fm = new_fm.add_prefix('여성_')

In [196]:
all_ = pd.merge(new_fm, new_m, on='영화제목', how='outer')

In [197]:
all_['평점평균비교'] = all_[('여성_평점','여성_mean')]-all_['남성_평점','남성_mean']

In [198]:
all_ = all_.dropna(axis=0)

In [207]:
l = list(range(0,29))
l

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28]

In [208]:
for i in l:
    if all_.평점평균비교[i]<0:
        print("남성이 좋아합니다")
    else:
        print("여성이 좋아합니다.")

남성이 좋아합니다
여성이 좋아합니다.
남성이 좋아합니다
여성이 좋아합니다.
남성이 좋아합니다
남성이 좋아합니다
여성이 좋아합니다.
남성이 좋아합니다
남성이 좋아합니다
남성이 좋아합니다
남성이 좋아합니다
남성이 좋아합니다
남성이 좋아합니다
남성이 좋아합니다
여성이 좋아합니다.
여성이 좋아합니다.
남성이 좋아합니다
남성이 좋아합니다
남성이 좋아합니다
여성이 좋아합니다.
여성이 좋아합니다.
남성이 좋아합니다
남성이 좋아합니다
여성이 좋아합니다.
남성이 좋아합니다
남성이 좋아합니다
남성이 좋아합니다
여성이 좋아합니다.
여성이 좋아합니다.


## [실습 #4] 연령대 별로 영화 평점 분석하기
연령대(10대 미만, 10대, 20대, ...50대) 컬럼을 추가한 후, 영화별 연령대별 영화평점 구하기

# 정답

In [373]:
def calculate_ages(x):
    if x <10:
        return '10대 미만'
    elif x <20:
        return '10대'
    elif x <30:
        return '20대'
    elif x <40:
        return '30대'
    elif x <50:
        return '40대'
    else:
        return '50대 이상'

In [377]:
# 같은 결과
data['연령대'] = data.연령.apply(calculate_ages)
data.연령.apply(lambda x: calculate_ages(x))

In [384]:
ex4 = data.pivot_table(index='영화제목',columns = '연령대', values='평점',aggfunc='mean')

In [385]:
ex4

연령대,10대,10대 미만,20대,30대,40대,50대 이상
영화제목,,,,,,
"$1,000,000 Duck (1971)",3.000000,NaN,3.090909,3.133333,2.000000,2.750000
'Night Mother (1986),4.666667,2.000000,3.423077,2.904762,3.833333,3.750000
'Til There Was You (1997),2.500000,3.500000,2.666667,2.900000,2.333333,2.600000
"'burbs, The (1989)",3.244444,4.500000,2.652174,2.818182,2.545455,3.100000
...And Justice for All (1979),3.428571,3.000000,3.724138,3.657143,4.100000,3.674419
...,...,...,...,...,...,...
"Zed & Two Noughts, A (1985)",3.000000,1.000000,3.375000,3.777778,4.000000,3.000000
Zero Effect (1998),3.883333,4.125000,3.715278,3.608696,3.764706,3.769231
Zero Kelvin (Kjærlighetens kjøtere) (1995),NaN,NaN,NaN,3.500000,NaN,NaN
